In [5]:
import pandas as pd

In [25]:
data_file = "./solver-benchmark/results/benchmark_results.csv"  # NOTE relative path!
df = pd.read_csv(data_file)
df

,Benchmark,Solver,Status,Termination Condition,Objective Value,Runtime (s),Memory Usage (MB)
0,pypsa-wind+sol+ely-1h-ucwind,gurobi,ok,optimal,3.139636e+11,5.985078,408.528
1,pypsa-wind+sol+ely-1h-ucwind,gurobi,ok,optimal,3.139636e+11,5.521643,396.212
2,pypsa-wind+sol+ely-1h-ucwind,gurobi,ok,optimal,3.139636e+11,5.608232,396.228
3,pypsa-wind+sol+ely-1h-ucwind,gurobi,ok,optimal,3.139636e+11,5.632786,401.760
4,pypsa-wind+sol+ely-1h-ucwind,gurobi,ok,optimal,3.139636e+11,5.560092,405.600
...,...,...,...,...,...,...,...
75,pypsa-wind+sol+ely-1h,scip,ok,optimal,8.466753e+10,209.339396,719.224
76,pypsa-wind+sol+ely-1h,scip,ok,optimal,8.466753e+10,209.296401,717.900
77,pypsa-wind+sol+ely-1h,scip,ok,optimal,8.466753e+10,209.478274,721.768
78,pypsa-wind+sol+ely-1h,scip,ok,optimal,8.466753e+10,210.108862,719.096
